# Bert experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import AutoModel, BertTokenizerFast,BertTokenizer, DistilBertTokenizerFast
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM

In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_4.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Bert

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

20


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.910197,0.144330,0.072103,0.166667,0.097030
2,No log,2.749718,0.134021,0.056411,0.194167,0.081265
3,No log,2.244664,0.412371,0.356984,0.472857,0.367099
4,No log,1.787238,0.525773,0.503571,0.577976,0.510929
5,No log,1.416470,0.649485,0.636329,0.642262,0.611235
6,No log,1.163643,0.690722,0.726548,0.680595,0.664639
7,No log,1.101902,0.649485,0.648214,0.661905,0.638407
8,No log,1.058184,0.701031,0.717381,0.719048,0.681453
9,No log,1.223633,0.628866,0.681667,0.671786,0.636235
10,1.447000,1.091778,0.721649,0.754167,0.720357,0.718861


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=660, training_loss=1.1043037132783369, metrics={'train_runtime': 318.2861, 'train_samples_per_second': 32.838, 'train_steps_per_second': 2.074, 'total_flos': 2750481197088768.0, 'train_loss': 1.1043037132783369, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.59375   , -0.3972168 , -0.4152832 , ..., -0.63964844,
         4.0390625 ,  0.01473236],
       [-0.21569824, -1.1777344 , -0.25341797, ..., -1.1699219 ,
         2.3789062 ,  0.72509766],
       [-1.0253906 , -0.3779297 , -1.2363281 , ..., -0.6020508 ,
         0.51416016,  5.96875   ],
       ...,
       [-1.0332031 , -0.3474121 , -0.8930664 , ..., -1.0302734 ,
         3.953125  ,  5.4296875 ],
       [-1.234375  , -1.5273438 , -0.94970703, ...,  0.7519531 ,
        -0.5175781 ,  1.1171875 ],
       [-1.3544922 , -0.7133789 , -0.25805664, ..., -0.9604492 ,
         6.734375  ,  1.4248047 ]], dtype=float32), label_ids=array([16,  3, 19, 16,  7, 18, 15,  4, 17,  2, 11,  5, 10,  0, 11, 16,  9,
       15,  9,  0, 10, 11,  5, 13,  8,  1, 10, 13,  0,  0,  8, 13,  8, 19,
       13,  2, 14,  4,  5,  0,  5,  7, 12, 10,  5, 16,  6, 16,  2,  1, 10,
        9, 12, 11,  6,  6,  4,  0,  3, 10,  4,  4, 19,  0, 13, 17, 13,  5,
        3,  7, 15, 19, 16,  2,  

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.975052,0.051546,0.002976,0.050000,0.005618
2,No log,2.828548,0.226804,0.141798,0.256190,0.156237
3,No log,2.012172,0.453608,0.464345,0.489405,0.404782
4,No log,1.668214,0.484536,0.514336,0.554048,0.467550
5,No log,1.553115,0.597938,0.702427,0.608690,0.592561
6,No log,1.133549,0.659794,0.694067,0.660119,0.632711
7,No log,1.127272,0.670103,0.724603,0.666548,0.655996
8,No log,0.953724,0.742268,0.730873,0.740000,0.725333
9,No log,1.235761,0.701031,0.679464,0.713452,0.665708
10,1.282400,1.383116,0.680412,0.727073,0.688929,0.664590


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=660, training_loss=0.9742307390227463, metrics={'train_runtime': 332.2907, 'train_samples_per_second': 31.454, 'train_steps_per_second': 1.986, 'total_flos': 2750481197088768.0, 'train_loss': 0.9742307390227463, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.4423828 , -1.0107422 , -0.44555664, ..., -0.98291016,
         1.9453125 , -0.22338867],
       [ 0.28320312, -1.0527344 , -0.9838867 , ..., -1.0703125 ,
         3.0566406 , -0.38427734],
       [-0.70654297, -2.1894531 , -0.42211914, ..., -0.6777344 ,
        -0.2578125 ,  7.5195312 ],
       ...,
       [-1.1738281 , -1.4072266 , -0.6484375 , ..., -0.52685547,
         7.7773438 ,  0.88134766],
       [-1.7353516 ,  0.15161133, -0.79345703, ...,  0.5097656 ,
        -0.15734863,  0.3395996 ],
       [-0.92529297, -1.7539062 , -0.94433594, ..., -0.9609375 ,
         7.5       ,  0.86083984]], dtype=float32), label_ids=array([16,  3, 19, 16,  7, 18, 15,  4, 17,  2, 11,  5, 10,  0, 11, 16,  9,
       15,  9,  0, 10, 11,  5, 13,  8,  1, 10, 13,  0,  0,  8, 13,  8, 19,
       13,  2, 14,  4,  5,  0,  5,  7, 12, 10,  5, 16,  6, 16,  2,  1, 10,
        9, 12, 11,  6,  6,  4,  0,  3, 10,  4,  4, 19,  0, 13, 17, 13,  5,
        3,  7, 15, 19, 16,  2,  